Let's investigate the conjecture on elliptic periods.

We want to find a counter-example, i.e., we want to find a curve over $\mathbb{F}_p$ such that a subgroup of order $ℓ$ is defined over an extension of degree $r|(ℓ-1)$, and such that the periods of that subgroup fall in a field smaller than $\mathbb{F}_{p^r}$.

To avoid pitfalls, we prefer having $r>2$, $ℓ≠p$, and maybe even $r≠p$. Since we have to mod-out the automorphisms of the curve, the smallest possible $ℓ$ to exhibit a counter-example is $13$, so that $(ℤ/13ℤ)^*/\{±1\}=\mathcal{C}_2\times\mathcal{C}_3$.

So, let's set $ℓ=13$, $r=3$. The elliptic period is defined as $x(P)+x([5]P)$ for $P$ a point of order 13 in a well chosen subgroup.

We look for a curve that has $13$-torsion points in an extension of degree $3$.

Here's an idea to generate tons of examples: take a curve over $ℚ$ with this property, and reduce modulo many primes. We are lucky: there are infinitely many such curves (<http://arxiv.org/pdf/1211.2188.pdf>, prop 31), and one such example is curve `147b1` in Cremona's database.

In [1]:
E = EllipticCurve('147b1')
E

Elliptic Curve defined by y^2 + y = x^3 + x^2 - 114*x + 473 over Rational Field

The curve is singular at $3$ and $7$

In [2]:
E.discriminant().factor()

-1 * 3 * 7^8

And certainly supersingular at $2$

In [3]:
E.j_invariant(), E.j_invariant().factor()

(-28672/3, -1 * 2^12 * 3^-1 * 7)

It's $13$-division polynomial factors as expected

In [4]:
[f[0].degree() for f in E.division_polynomial(13).factor()]

[3, 3, 78]

In [5]:
facts = E.division_polynomial(13).factor()[:2]
facts

[(x^3 - 48*x^2 + 425*x - 1009, 1), (x^3 + x^2 - 114*x - 127, 1)]

Let's look at multiplication by $5$. It's action on the abscissa is given by a rational fraction in $x$.

In [6]:
mul5 = E.multiplication_by_m(5)[0]
mul5

(x^25 + 11400*x^23 - 952440*x^22 + 21459390*x^21 - 109428732*x^20 - 4950010695*x^19 + 144784682400*x^18 - 181243777785*x^17 - 76264253249145*x^16 + 1852228009144500*x^15 - 17751704371327800*x^14 - 13405942951098075*x^13 + 2174244297003851940*x^12 - 23804139576831231690*x^11 + 121433153419916125074*x^10 - 449464862928041129205*x^9 + 5800247633495175127650*x^8 - 83614774727436641897175*x^7 + 721566408459768769400580*x^6 - 4069574415747167715056277*x^5 + 16123970604415307494932945*x^4 - 46252779384036854092437225*x^3 + 94273994056986749224110510*x^2 - 123535079543891692412564130*x + 77552624336221526363279505)/(25*x^24 + 200*x^23 - 70120*x^22 + 1425070*x^21 + 48308554*x^20 - 2610219360*x^19 + 43594684875*x^18 - 67903799190*x^17 - 9897608934825*x^16 + 202483959479556*x^15 - 2037974926602780*x^14 + 6054274701348300*x^13 + 169194401041110165*x^12 - 3540471811324712460*x^11 + 39604874743194363378*x^10 - 306463233082971903660*x^9 + 1678785952137330965175*x^8 - 5689479606125098303260*x^7 + 3210

As we expect, multiplication by $5$ sends the abscissas of one factor to the other. We can verify this with a resultant.

In [7]:
_.<x,y> = QQ[]
facts[0][0].resultant(mul5.denominator()*y - mul5.numerator()).univariate_polynomial().monic() == facts[1][0](y)

True

In [8]:
facts[1][0].resultant(mul5.denominator()*y - mul5.numerator()).univariate_polynomial().monic() == facts[0][0](y)

True

By the same resultant trick, we get the minimal polynomial of the elliptic period. Remember: the period is defined as $x(P)+x([5]P)$.

In [9]:
period = facts[0][0].resultant(mul5.denominator()*(y - x) - mul5.numerator()).univariate_polynomial()
period

1347137238494276547832006567721872890819326613454654477690085519113574118965817601*y^3 - 63315450209230997748104308682928025868508350832368760451434019398337983591393427247*y^2 + 375851289539903156845129832394402536538592125153848599275533859832687179191463110679*y + 5430310208370428764310818474486869622892705578835712199568734727546817273551210749631

Quite big coefficients, but let's look at the leading one

In [10]:
period.leading_coefficient().factor()

7^96

Ahah! Only the singular prime appears in it! Of course not a coincidence.

Let's simplify

In [11]:
period = period.monic()
period

y^3 - 47*y^2 + 279*y + 4031

Surprisingly short!

Just to be sure, let's check that this does not depend on the choice of the initial point.

In [12]:
facts[1][0].resultant(mul5.denominator()*(y - x) - mul5.numerator()).univariate_polynomial().monic() == period

True

So, now we look for a prime $p≠2,3,7$ such that the $13$-torsion points live in $\mathbb{F}_{p^3}$, but the period lives in $\mathbb{F}_p$.

We can read this directly on the polynomials.

In [13]:
def test(p):
    R = PolynomialRing(GF(p), 'x')
    return R(facts[0][0]).roots(), R(facts[1][0]).roots(), R(period).roots()

[(p,test(p)) for p in primes(100)]

[(2, ([], [], [(1, 3)])),
 (3, ([], [], [])),
 (5, ([], [], [])),
 (7, ([(2, 3)], [(2, 3)], [(4, 3)])),
 (11, ([], [], [])),
 (13,
  ([(10, 1), (8, 1), (4, 1)],
   [(12, 1), (7, 1), (6, 1)],
   [(4, 1), (3, 1), (1, 1)])),
 (17, ([], [], [])),
 (19, ([], [], [])),
 (23, ([], [], [])),
 (29,
  ([(27, 1), (15, 1), (6, 1)],
   [(23, 1), (22, 1), (12, 1)],
   [(0, 1), (27, 1), (20, 1)])),
 (31, ([], [], [])),
 (37, ([], [], [])),
 (41,
  ([(24, 1), (19, 1), (5, 1)],
   [(18, 1), (15, 1), (7, 1)],
   [(26, 1), (20, 1), (1, 1)])),
 (43,
  ([(31, 1), (10, 1), (7, 1)],
   [(21, 1), (15, 1), (6, 1)],
   [(28, 1), (16, 1), (3, 1)])),
 (47, ([], [], [])),
 (53, ([], [], [])),
 (59, ([], [], [])),
 (61, ([], [], [])),
 (67, ([], [], [])),
 (71,
  ([(69, 1), (35, 1), (15, 1)],
   [(30, 1), (29, 1), (11, 1)],
   [(64, 1), (28, 1), (26, 1)])),
 (73, ([], [], [])),
 (79, ([], [], [])),
 (83,
  ([(61, 1), (51, 1), (19, 1)],
   [(72, 1), (69, 1), (24, 1)],
   [(50, 1), (43, 1), (37, 1)])),
 (89, ([], [],

Bad luck! Or maybe not.

One thing is obvious: if the first two split, the third splits as well. The opposite is not so obvious, but there might be an easy explanation too.

I'm testing for all (pseudo)-primes right now. No luck up to 6 millions :(

Ok. Let's try with norms instead of traces.

In [14]:
period2 = facts[0][0].resultant(mul5.denominator()*y - x*mul5.numerator()).univariate_polynomial()
period2

1347137238494276547832006567721872890819326613454654477690085519113574118965817601*y^3 + 21554195815908424765312105083549966253109225815274471643041368305817185903453081616*y^2 - 5583883853558776290763667223207163132446108812769542810025404476725764723113313956145*y - 172626207152372079668836817607583957848260970227919788734640628675770728326636764844943

In [15]:
period2.leading_coefficient().factor()

7^96

In [16]:
period2 = period2.monic()
period2

y^3 + 16*y^2 - 4145*y - 128143

In [17]:
facts[1][0].resultant(mul5.denominator()*y - x*mul5.numerator()).univariate_polynomial().monic() == period2

True

In [18]:
def test2(p):
    R = PolynomialRing(GF(p), 'x')
    return R(facts[0][0]).roots(), R(facts[1][0]).roots(), R(period2).roots()

[(p,test2(p)) for p in primes(100)]

[(2, ([], [], [])),
 (3, ([], [], [])),
 (5, ([], [], [])),
 (7, ([(2, 3)], [(2, 3)], [(4, 3)])),
 (11, ([], [], [])),
 (13,
  ([(10, 1), (8, 1), (4, 1)], [(12, 1), (7, 1), (6, 1)], [(5, 1), (9, 2)])),
 (17, ([], [], [])),
 (19, ([], [], [])),
 (23, ([], [], [])),
 (29,
  ([(27, 1), (15, 1), (6, 1)],
   [(23, 1), (22, 1), (12, 1)],
   [(22, 1), (14, 1), (6, 1)])),
 (31, ([], [], [])),
 (37, ([], [], [])),
 (41,
  ([(24, 1), (19, 1), (5, 1)],
   [(18, 1), (15, 1), (7, 1)],
   [(34, 1), (22, 1), (10, 1)])),
 (43,
  ([(31, 1), (10, 1), (7, 1)],
   [(21, 1), (15, 1), (6, 1)],
   [(35, 1), (18, 1), (17, 1)])),
 (47, ([], [], [])),
 (53, ([], [], [])),
 (59, ([], [], [])),
 (61, ([], [], [])),
 (67, ([], [], [])),
 (71,
  ([(69, 1), (35, 1), (15, 1)],
   [(30, 1), (29, 1), (11, 1)],
   [(23, 1), (21, 1), (11, 1)])),
 (73, ([], [], [])),
 (79, ([], [], [])),
 (83,
  ([(61, 1), (51, 1), (19, 1)],
   [(72, 1), (69, 1), (24, 1)],
   [(76, 1), (41, 1), (33, 1)])),
 (89, ([], [], [])),
 (97,
  ([(

Ok. Something weird is happening for $p=13$, but nothing remarkable. Here too, no luck up to several millions.

So, what does this tell us on the conjecture? Not much!

Maybe it is easy to explain why if the trace/norm splits, then the torsion polynomials split too. Maybe this can open a path to show the conjecture. Or maybe not: after all, we may think of all these examples as the same curve.

Maybe using more curves from the infinite family in <http://arxiv.org/pdf/1211.2188.pdf> can lead to a counter-example. Or maybe not.

Whatever is true, more tought is needed.